In [0]:
# Long Short-term Memory method
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from __future__ import division

import warnings
warnings.filterwarnings("ignore")

#import Keras
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [69]:
df = pd.read_csv('https://raw.githubusercontent.com/RHaile/final_project/master/data/CleanedBorderdata_2008-2018.csv', parse_dates=["Date"])
df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Latitutde,Longitude
0,Van Buren,Maine,108,US-Canada Border,2018-12-01,Trucks,1204,-67.94271,47.16207
1,Maida,North Dakota,3416,US-Canada Border,2018-12-01,Trucks,170,-98.36953,48.98568
2,Douglas,Arizona,2601,US-Mexico Border,2018-12-01,Buses,210,-109.54472,31.34444
3,Presidio,Texas,2403,US-Mexico Border,2018-12-01,Bus Passengers,238,-104.37167,29.56056
4,Anacortes,Washington,3010,US-Canada Border,2018-12-01,Personal Vehicle Passengers,5350,-122.61739,48.49988


In [75]:
df['Date'] = df['Date'].dt.year.astype('str') + '-' + df['Date'].dt.month.astype('str') + '-01'
df['Date'] = pd.to_datetime(df['Date'])

df = df.groupby('Date').Value.sum().reset_index()
df.head()

,Date,Value
0,2008-01-01,32018021
1,2008-02-01,30452440
2,2008-03-01,34743026
3,2008-04-01,32216753
4,2008-05-01,34651654


In [119]:
# Getting difference between value from last month - since it has pattern by month
df_diff = df.copy()
df_diff['Prev_value'] = df_diff['Value'].shift(1)
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['Value'] - df_diff['Prev_value'])
df_diff.tail(10)

,Date,Value,Prev_value,diff
122,2018-03-01,32458891,27400251.0,5058640.0
123,2018-04-01,30949588,32458891.0,-1509303.0
124,2018-05-01,32646367,30949588.0,1696779.0
125,2018-06-01,32105809,32646367.0,-540558.0
126,2018-07-01,35056685,32105809.0,2950876.0
127,2018-08-01,35170600,35056685.0,113915.0
128,2018-09-01,31105680,35170600.0,-4064920.0
129,2018-10-01,31669698,31105680.0,564018.0
130,2018-11-01,29560756,31669698.0,-2108942.0
131,2018-12-01,31316998,29560756.0,1756242.0


In [83]:
# Transform time series
df_feature = df_diff.drop(['Prev_value'],axis=1)
for inc in range(1,13):
    field_name = 'lag_' + str(inc)
    df_feature[field_name] = df_feature['diff'].shift(inc)
df_feature = df_supervised.dropna().reset_index(drop=True)
df_feature.head()

,Date,Value,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
0,2009-02-01,26121160,-3047781.0,-1849702.0,273012.0,-2040998.0,-102534.0,-4752177.0,800107.0,2943733.0,-754154.0,2434901.0,-2526273.0,4290586.0,-1565581.0
1,2009-03-01,29745132,3623972.0,-3047781.0,-1849702.0,273012.0,-2040998.0,-102534.0,-4752177.0,800107.0,2943733.0,-754154.0,2434901.0,-2526273.0,4290586.0
2,2009-04-01,29014657,-730475.0,3623972.0,-3047781.0,-1849702.0,273012.0,-2040998.0,-102534.0,-4752177.0,800107.0,2943733.0,-754154.0,2434901.0,-2526273.0
3,2009-05-01,30443253,1428596.0,-730475.0,3623972.0,-3047781.0,-1849702.0,273012.0,-2040998.0,-102534.0,-4752177.0,800107.0,2943733.0,-754154.0,2434901.0
4,2009-06-01,29477132,-966121.0,1428596.0,-730475.0,3623972.0,-3047781.0,-1849702.0,273012.0,-2040998.0,-102534.0,-4752177.0,800107.0,2943733.0,-754154.0


In [87]:
# Adjusted R-squared
import statsmodels.formula.api as smf
# Regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_feature)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

#lag_1 - 3% variation

0.08861388135179249


In [88]:
# Adjusted R-squared
import statsmodels.formula.api as smf
# Regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5 + lag_6 + lag_7 + lag_8 + lag_9 + lag_10 + lag_11 + lag_12', data=df_feature)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.9196401040935578


In [0]:
from sklearn.preprocessing import MinMaxScaler
df_model = df_feature.drop(['Value','Date'],axis=1)
# Predicting 24 months
train_set, test_set = df_model[0:-24].values, df_model[-24:].values

In [0]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler = scaler.fit(train_set)

train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)

test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [0]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [94]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, nb_epoch=1, batch_size=1, verbose=1, shuffle=False)

Epoch 1/1
95/95 [==============================] - 2s 19ms/step - loss: 0.3022


In [98]:
y_predict = model.predict(X_test,batch_size=1)
y_predict

array([[-0.0495544 ],
       [-0.07314156],
       [-0.12451997],
       [ 0.05900397],
       [ 0.02579458],
       [-0.08767343],
       [ 0.01927588],
       [ 0.06081104],
       [ 0.11528247],
       [-0.1117842 ],
       [-0.16026416],
       [ 0.05867553],
       [-0.03703596],
       [-0.12352567],
       [-0.16093087],
       [ 0.07017492],
       [ 0.00631452],
       [-0.10546456],
       [ 0.0194139 ],
       [ 0.04665505],
       [ 0.09452103],
       [-0.12696466],
       [-0.1677821 ],
       [ 0.06164027]], dtype=float32)

In [101]:
y_predict = y_predict.reshape(y_predict.shape[0], 1, y_predict.shape[1])

predict_test_set = []
for index in range(0,len(y_predict)):
    print (np.concatenate([y_predict[index],X_test[index]],axis=1))
    predict_test_set.append(np.concatenate([y_predict[index],X_test[index]],axis=1))

predict_test_set = np.array(predict_test_set)
predict_test_set = predict_test_set.reshape(predict_test_set.shape[0], predict_test_set.shape[2])

predict_test_set_inverted = scaler.inverse_transform(predict_test_set)

[[-0.0495544   0.42280053 -0.21405126  0.27186536 -0.54526736  0.01979298
   0.85422363 -0.02061583  0.5393657  -0.11000251  0.82019817 -0.22894732
  -0.36759887]]
[[-0.07314156 -0.29895354  0.42280053 -0.21405126  0.27186536 -0.54526736
   0.01979298  0.85422363 -0.02061583  0.5393657  -0.11000251  0.82019817
  -0.22894732]]
[[-0.12451997 -0.42065707 -0.29895354  0.42280053 -0.21405126  0.27186536
  -0.54526736  0.01979298  0.85422363 -0.02061583  0.5393657  -0.11000251
   0.82019817]]
[[ 0.05900397  1.03830027 -0.42065707 -0.29895354  0.42280053 -0.21405126
   0.27186536 -0.54526736  0.01979298  0.85422363 -0.02061583  0.5393657
  -0.11000251]]
[[ 0.02579458  0.15317161  1.03830027 -0.42065707 -0.29895354  0.42280053
  -0.21405126  0.27186536 -0.54526736  0.01979298  0.85422363 -0.02061583
   0.5393657 ]]
[[-0.08767343  0.25581759  0.15317161  1.03830027 -0.42065707 -0.29895354
   0.42280053 -0.21405126  0.27186536 -0.54526736  0.01979298  0.85422363
  -0.02061583]]
[[ 0.01927588  0.

In [123]:
result_list = []
date = list(df[-24:].Date)
Actual = list(df[-24:].Value)
for index in range(0,len(predict_test_set_inverted)):
    result_dict = {}
    result_dict['date'] = date[index]
    result_dict['Prediction'] = int(predict_test_set_inverted[index][0] + Actual[index])
    result_dict['Actual'] = int(Actual[index])
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)
df_result

,date,Prediction,Actual
0,2017-01-01,27951508,28826910
1,2017-02-01,25087430,26081678
2,2017-03-01,29434398,30687520
3,2017-04-01,30505148,30833570
4,2017-05-01,31001060,31496810
5,2017-06-01,30049504,31116972
6,2017-07-01,34320316,34848911
7,2017-08-01,34125067,34444384
8,2017-09-01,30637079,30681937
9,2017-10-01,30425049,31614001
